In [ ]:
import boto3

bedrock_agent = boto3.client("bedrock-agent")
bedrock_runtime = boto3.client("bedrock-agent-runtime")

model_arn = "arn:aws:bedrock:eu-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
knowledge_base_id = "ZTPJ2E7RTK"  # Your Knowledge Base ID

response = bedrock_runtime.retrieve_and_generate(
    input={"text": "What is the carcinogenic pottential of Omeprazole?"},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": knowledge_base_id,
            "modelArn": model_arn
            # You can add an optional "generationConfiguration" dict here if needed for custom settings
        }
    },
    #sessionId="user123-session-001"
)

print(response["output"]["text"])

citations = response.get("citations", [])
for citation in citations:
    for ref in citation.get("retrievedReferences", []):
        print(ref["location"]["s3Location"]["uri"])
session_id = response.get("sessionId")
print("New session started:", session_id)

In [ ]:
def continue_conversation(question, session_id=None):
    import boto3
    brt = boto3.client("bedrock-agent-runtime", region_name="eu-west-2")
    # Base request
    request = {
        "input": {"text": question},
        "retrieveAndGenerateConfiguration": {
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": "ZTPJ2E7RTK",
                "modelArn": "arn:aws:bedrock:eu-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
            }
        }
    }
    # Add sessionId only if it's provided
    if session_id:
        request["sessionId"] = session_id
    response = brt.retrieve_and_generate(**request)
    # Print the generated answer
    print("\n🧠 Answer:\n")
    print(response["output"]["text"])
    # Print citations
    print("\n📚 Citations:\n")
    citations = response.get("citations", [])
    if not citations:
        print("No citations found.")
    else:
        for c in citations:
            for ref in c.get("retrievedReferences", []):
                uri = ref.get("location", {}).get("s3Location", {}).get("uri", "")
                snippet = ref.get("content", {}).get("text", "")
                print(f"- {uri}")
                if snippet:
                    print(f"  → Snippet: {snippet[:200]}...\n")
    # Show session ID (in case it's new)
    print("\n🧾 Session ID:", response.get("sessionId", "[not returned]"))

In [ ]:
answer = continue_conversation("compare it with Pantoprazole!", "5e6af1ea-c376-4fdc-a43e-c41133efbdcb")
print(answer)

🧠 Answer:

Based on the search results, there is some evidence that both omeprazole and pantoprazole may have carcinogenic potential in rodents, but their carcinogenic potential in humans is not definitively established. Omeprazole has been reported to induce unscheduled DNA synthesis in the gastric mucosa of rodents, suggesting it may be a potential genotoxic carcinogen, possibly due to an unstable sulphenimide metabolite (source 2). Pantoprazole has shown positive carcinogenic effects in rats and mice, causing stomach, liver, and thyroid tumors in rodent studies (source 4). However, the evidence regarding carcinogenicity in humans is less clear. Source 4 notes that FDA labels for both omeprazole and pantoprazole include a precaution about potential gastric malignancy, but definitive conclusions cannot be drawn. More studies are needed to determine if the carcinogenic effects observed in rodents translate to humans taking these drugs (source 1).

📚 Citations:

- s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/1795642.txt
  → Snippet: Because of its reported ability to induce unscheduled DNA synthesis in the gastric mucosa, the safety of omeprazole, a potentially clinically useful anti-ulcer drug, has been the subject of debate. We...

- s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Computational Toxicology/Stalford et al. - 2024 - Structuring expert review using AOPs Enabling rob.pdf
  → Snippet: Table 2 Carcinogenicity data for compounds pharmacologically similar to Lansoprazole.     Similar Compound Carcinogenicity FDA Labels/Reviews     Omeprazole Rat – stomach (gastric mucosa) [21,41,42] G...

- s3://lhasalimited-gen-ai-unstructured-data-store-transform/pubmed/25983631.pdf
  → Snippet: c C on     si de     ri ng     th e     m w     h cl     on es     fo r     th e     si ng     le sp     ot s     an d     m w     h fo     r th     e tw     in sp     ot s.     d F re     qu en     c...

- s3://lhasalimited-gen-ai-unstructured-data-store-transform/zotero-papers/Computational Toxicology/Stalford et al. - 2024 - Structuring expert review using AOPs Enabling rob.pdf
  → Snippet: Table 2 Carcinogenicity data for compounds pharmacologically similar to Lansoprazole.     Similar Compound Carcinogenicity FDA Labels/Reviews     Omeprazole Rat – stomach (gastric mucosa) [21,41,42] G...